In [1]:
# Load libraries

import pandas as pd
import slippi as slp
import os

In [2]:
# Load data

data_dir = "./data/"
games = []

for path in os.listdir(data_dir):
    full_path = os.path.join(data_dir, path)
    if os.path.isfile(full_path):
        games.append(full_path)

games[0:10]

['./data/Game_20210117T162123.slp',
 './data/Game_20210117T191157.slp',
 './data/Game_20210117T161328.slp',
 './data/Game_20210117T203707.slp',
 './data/Game_20210117T194724.slp',
 './data/Game_20210117T192619.slp',
 './data/Game_20210117T164621.slp',
 './data/Game_20210117T200301.slp',
 './data/Game_20210117T201707.slp',
 './data/Game_20210117T184252.slp']

In [3]:
# Code from Raffy Santayana's Medium post at https://medium.com/analytics-vidhya/parsing-slippi-files-into-pandas-dataframes-the-metadata-2efdfcb8562c

def metadata_to_df(slp_paths):
    '''
    Of a collection of games, store the metadata as a dataframe.
    
    slp_paths (list): each value is the file path to games
    returns a dataframe
    '''
    # Used to determine how far along the function is while waiting
    length = len(slp_paths)
    count = 0
    
    # lists of values to populate dataframe.
    dates, game_id, durations, plats, num_players, p1_chars, p2_chars, \
    p3_chars, p4_chars, stages, is_teams, is_pal = [], [], [], [], [], \
    [], [], [], [], [], [], []
    
    # For each filepath in the provided list of filepaths
    for path in slp_paths:
        count += 1
        print(f'Parsing metadata from file {count} of {length}: {round(count / length * 100, 2)}%', end = '\r')
        
        # try to instantiate the Game object, else skip it and try the next one
        try:
            game = slp.Game(path)
        except:
            print(f'Skip game {count} of {length}')
            continue
            
        # set game ID
        # to get file path using game_id:
        # ../folder_directory/Game_[game_id].slp
        ident_strip = slp_paths[count - 1].split('/')
        game_id.append(ident_strip[-2] + ident_strip[-1].strip('Game_').strip('.slp'))
        
        # take the date, duration, and platform data
        dates.append(game.metadata.date)
        durations.append(game.metadata.duration)
        plats.append(game.metadata.platform)# get active ports
        ports = [game.start.players.index(port) if port != None else None for port in game.start.players]# get characters
        characters = [game.start.players[port].character if port != None else port for port in ports]
        p1_chars.append(characters[0])
        p2_chars.append(characters[1])
        p3_chars.append(characters[2])
        p4_chars.append(characters[3])
        
        # get stages played on
        stages.append(game.start.stage)
        
        # is the game not a 1v1
        is_teams.append(game.start.is_teams)
        
        # is this not a v1.02 match
        is_pal.append(game.start.is_pal)
    
    # return metadata DataFrame
    return pd.DataFrame(data = {
            'game_id': game_id,
            'date': dates,
            'duration': durations,
            'platform': plats,
            'p1_char': p1_chars,
            'p2_char': p2_chars,
            'p3_char': p3_chars,
            'p4_char': p4_chars,
            'stage': stages,
            'is_teams': is_teams,
            'is_pal': is_pal
        })

In [4]:
df = metadata_to_df(games)

In [5]:
# A dictionary to map CSSCharacter values to their respective character names
csscharacter = {
    0: 'Captain Falcon',
    1: 'Donkey Kong',
    2: 'Fox',
    3: 'Game and Watch',
    4: 'Kirby',
    5: 'Bowser',
    6: 'Link',
    7: 'Luigi',
    8: 'Mario',
    9: 'Marth',
    10: 'Mewtwo',
    11: 'Ness',
    12: 'Peach',
    13: 'Pikachu',
    14: 'Ice Climbers',
    15: 'Jigglypuff',
    16: 'Samus',
    17: 'Yoshi',
    18: 'Zelda',
    19: 'Sheik',
    20: 'Falco',
    21: 'Young Link',
    22: 'Dr. Mario',
    23: 'Roy',
    24: 'Pichu',
    25: 'Ganondorf'
}# A dictionary to map stage values to their respective stage names
stages = {
    2: 'Fountain of Dreams',
    3: 'Pokemon Stadium',
    4: "Princess Peach's Castle",
    5: 'Kongo Jungle',
    6: 'Brinstar',
    7: 'Corneria',
    8: "Yoshi's Story",
    9: 'Onett',
    10: 'Mute City',
    11: 'Rainbow Cruise',
    12: 'Jungle Japes',
    13: 'Great Bay',
    14: 'Hyrule Temple',
    15: 'Brinstar Depths',
    16: "Yoshi's Island",
    17: 'Green Greens',
    18: 'Fourside',
    19: 'Mushroom Kingdom I',
    20: 'Mushroom Kingdom II',
    22: 'Venom',
    23: 'Poke Floats',
    24: 'Big Blue',
    25: 'Icicle Mountain',
    26: 'Icetop',
    27: 'Flat Zone',
    28: 'Dream Land 64',
    29: "Yoshi's Island 64",
    30: 'Kongo Jungle 64',
    31: 'Battlefield',
    32: 'Final Destination'
}# Mapping values to their respective series
df['p1_char_name'] = df['p1_char'].map(csscharacter)
df['p2_char_name'] = df['p2_char'].map(csscharacter)
df['p3_char_name'] = df['p3_char'].map(csscharacter)
df['p4_char_name'] = df['p4_char'].map(csscharacter)
df['stage_name'] = df['stage'].map(stages)

In [6]:
df.head(10)

,game_id,date,duration,platform,p1_char,p2_char,p3_char,p4_char,stage,is_teams,is_pal,p1_char_name,p2_char_name,p3_char_name,p4_char_name,stage_name
0,data20210117T162123,2021-01-17 21:21:23+00:00,13058,Platform.DOLPHIN,12,2,None,None,3,False,False,Peach,Fox,NaN,NaN,Pokemon Stadium
1,data20210117T191157,2021-01-18 00:11:57+00:00,19260,Platform.DOLPHIN,12,12,None,None,28,False,False,Peach,Peach,NaN,NaN,Dream Land 64
2,data20210117T161328,2021-01-17 21:13:28+00:00,11119,Platform.DOLPHIN,12,2,None,None,31,False,False,Peach,Fox,NaN,NaN,Battlefield
3,data20210117T203707,2021-01-18 01:37:07+00:00,10376,Platform.DOLPHIN,20,12,None,None,31,False,False,Falco,Peach,NaN,NaN,Battlefield
4,data20210117T194724,2021-01-18 00:47:24+00:00,30879,Platform.DOLPHIN,12,13,None,None,32,False,False,Peach,Pikachu,NaN,NaN,Final Destination
5,data20210117T192619,2021-01-18 00:26:19+00:00,14331,Platform.DOLPHIN,12,2,None,None,32,False,False,Peach,Fox,NaN,NaN,Final Destination
6,data20210117T164621,2021-01-17 21:46:21+00:00,9113,Platform.DOLPHIN,12,2,None,None,31,False,False,Peach,Fox,NaN,NaN,Battlefield
7,data20210117T200301,2021-01-18 01:03:01+00:00,13076,Platform.DOLPHIN,12,13,None,None,32,False,False,Peach,Pikachu,NaN,NaN,Final Destination
8,data20210117T201707,2021-01-18 01:17:07+00:00,5717,Platform.DOLPHIN,20,12,None,None,3,False,False,Falco,Peach,NaN,NaN,Pokemon Stadium
9,data20210117T184252,2021-01-17 23:42:52+00:00,9866,Platform.DOLPHIN,12,0,None,None,31,False,False,Peach,Captain Falcon,NaN,NaN,Battlefield


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=61a9c9e2-89b3-4f40-a8ec-07d133b06da4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>